In [1]:
from line_solver import *
GlobalConstants.setVerbose(VerboseLevel.STD)

In [2]:
model = Network('model')

# Block 1: nodes
node1 = Delay(model, "Delay")
node2 = Queue(model, "Queue1", SchedStrategy.FCFS)
node3 = Queue(model, "Queue2", SchedStrategy.FCFS)
node2.setNumberOfServers(3)
node3.setNumberOfServers(3)

# Block 2: classes
jobclass1 = ClosedClass(model, "Class1", 2, node1, 0)
jobclass2 = ClosedClass(model, "Class2", 2, node1, 0)
jobclass3 = ClosedClass(model, "Class3", 2, node1, 0)
jobclass4 = ClosedClass(model, "Class4", 1, node1, 0)

node1.setService(jobclass1, Exp(1)) # (Delay,Class1)
node1.setService(jobclass2, Exp(1)) # (Delay,Class2)
node1.setService(jobclass3, Exp(10)) # (Delay,Class3)
node1.setService(jobclass4, Exp(1)) # (Delay,Class4)

node2.setService(jobclass1, Exp(1)) # (Queue1,Class1)
node2.setService(jobclass2, Erlang(1, 2)) # (Queue1,Class2)
node2.setService(jobclass3, Exp(10)) # (Queue1,Class3)
node2.setService(jobclass4, Exp(1)) # (Queue1,Class4)

node3.setService(jobclass1, Disabled.getInstance()) # (Queue2,Class1)
node3.setService(jobclass2, Disabled.getInstance()) # (Queue2,Class2)
node3.setService(jobclass3, Erlang(1, 2)) # (Queue2,Class3)
node3.setService(jobclass4, Exp(1)) # (Queue2,Class4)

# Block 3: topology
routingMatrix = model.initRoutingMatrix()

routingMatrix.set(jobclass1, jobclass1, node1, node2, 0.50)
routingMatrix.set(jobclass1, jobclass1, node2, node1, 1.00)
routingMatrix.set(jobclass1, jobclass1, node3, node1, 1.00)

routingMatrix.set(jobclass1, jobclass2, node1, node2, 0.50)

routingMatrix.set(jobclass2, jobclass2, node2, node1, 1.00)
routingMatrix.set(jobclass2, jobclass2, node3, node1, 1.00)

routingMatrix.set(jobclass2, jobclass1, node1, node2, 1.00)

routingMatrix.set(jobclass3, jobclass3, node1, node2, 0.25)
routingMatrix.set(jobclass3, jobclass3, node1, node3, 0.25)
routingMatrix.set(jobclass3, jobclass3, node2, node1, 1.00)
routingMatrix.set(jobclass3, jobclass3, node3, node1, 1.00)

routingMatrix.set(jobclass3, jobclass4, node1, node2, 0.50)

routingMatrix.set(jobclass4, jobclass4, node2, node1, 1.00)
routingMatrix.set(jobclass4, jobclass4, node3, node1, 1.00)

routingMatrix.set(jobclass4, jobclass3, node1, node2, 1.00)

model.link(routingMatrix)

In [4]:
solver = np.array([], dtype=object)
#solver = np.append(solver, SolverCTMC(model,'keep',True))
solver = np.append(solver, SolverJMT(model,'seed',23000,'verbose',True,'keep',True))
solver = np.append(solver, SolverSSA(model,'seed',23000,'verbose',False,'samples',10000))
solver = np.append(solver, SolverFluid(model))
solver = np.append(solver, SolverMVA(model))
solver = np.append(solver, SolverNC(model,'exact'))
#solver = np.append(solver, SolverMAM(model))
#solver = np.append(solver, LINE(model))

AvgTable = np.empty(len(solver), dtype=object)
for s in range(len(solver)):
    print(f'\nSOLVER: {solver[s].getName()}')
    AvgTable[s] = solver[s].getAvgTable()



SOLVER: SolverJMT
JMT Model: /tmp/workspace/jsim/16984906335142444519/model.jsim
JMT Analysis completed. Runtime: 2.21 seconds
   Station JobClass    QLen    Util   RespT  ResidT    ArvR    Tput
0    Delay   Class1  0.9624  0.9624  0.9955  0.6636  0.9759  0.9775
1    Delay   Class2  0.4710  0.4710  0.9964  0.3321  0.4934  0.4884
2    Delay   Class3  0.1332  0.1332  0.1000  0.0667  1.3475  1.3475
3    Delay   Class4  0.6728  0.6728  0.9911  0.3304  0.6810  0.6811
4   Queue1   Class1  1.4139  0.3323  1.4149  0.9433  0.9758  0.9780
5   Queue1   Class2  1.1780  0.3303  2.4004  0.8001  0.4883  0.4883
6   Queue1   Class3  0.5653  0.0337  0.5611  0.2806  1.0220  1.0216
7   Queue1   Class4  0.9404  0.2327  1.3560  0.4520  0.6795  0.6812
10  Queue2   Class3  0.6757  0.2252  1.9819  0.3303  0.3370  0.3382

SOLVER: SolverSSA
   Station JobClass    QLen    Util   RespT  ResidT    ArvR    Tput
0    Delay   Class1  0.9742  0.9742  1.0000  0.6667  0.9858  0.9742
1    Delay   Class2  0.4436  0.4436  

java.lang.RuntimeException: The exact method requires the model to have a product-form solution. This model does not have one. You can use Network.hasProductFormSolution() to check before running the solver.
	at jline.solvers.nc.NCRunner.run(NCRunner.java:78)
	at jline.solvers.nc.SolverNC.runAnalyzer(SolverNC.java:85)
	at jline.solvers.NetworkSolver.getAvg(NetworkSolver.java:351)
	at jline.solvers.NetworkSolver.getAvgTable(NetworkSolver.java:868)


java.lang.RuntimeException: java.lang.RuntimeException: Unable to compute results and therefore unable to print AvgTable.